In [2]:
"""
Generating ewt data for different cycles for all levels

N2 F1

"""

'\nGenerating ewt data for different cycles for all levels\n\nN2 F1\n\n'

In [3]:
pwd

'C:\\Users\\vinee\\Documents\\BTP'

In [5]:
cd resampled_dataset_f1_n2/

C:\Users\vinee\Documents\BTP\resampled_dataset_f1_n2


In [6]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
import os
import glob
import re

C:\Users\vinee\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [9]:
import numpy as np
#%EWT functions
def ewt_vineet(f,boundaries):
    # signal spectrum
    ff = np.fft.fft(f)
    ff = abs(ff[0:int(np.ceil(ff.size / 2))])  # one-sided magnitude

    #extend the signal by mirroring to deal with boundaries
    ltemp = int(np.ceil(f.size/2)) #to behave the same as matlab's round
    fMirr =  np.append(np.flip(f[0:ltemp-1],axis = 0),f)
    fMirr = np.append(fMirr,np.flip(f[-ltemp-1:-1],axis = 0))
    ffMirr = np.fft.fft(fMirr)
    # build the corresponding filter bank
    mfb = EWT_Meyer_FilterBank(boundaries, ffMirr.size)

    # filter the signal to extract each subband
    ewt = np.zeros(mfb.shape)
    for k in range(mfb.shape[1]):
        ewt[:, k] = np.real(np.fft.ifft(np.conjugate(mfb[:, k]) * ffMirr))
    ewt = ewt[ltemp - 1:-ltemp, :]

    return ewt


def EWT_Meyer_FilterBank(boundaries, Nsig):
    """
     =========================================================================
     function mfb=EWT_Meyer_FilterBank(boundaries,Nsig)

     This function generate the filter bank (scaling function + wavelets)
     corresponding to the provided set of frequency segments

     Input parameters:
       -boundaries: vector containing the boundaries of frequency segments (0
                    and pi must NOT be in this vector)
       -Nsig: signal length

     Output:
       -mfb: cell containing each filter (in the Fourier domain), the scaling
             function comes first and then the successive wavelets

     Author: Jerome Gilles
     Institution: UCLA - Department of Mathematics
     Year: 2012
     Version: 1.0

     Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
     Universidade Federal de Minas Gerais - Brasil
     Núcleo de Neurociências
     =========================================================================
     """
    Npic = len(boundaries)
    # compute gamma
    gamma = 1
    for k in range(Npic - 1):
        r = (boundaries[k + 1] - boundaries[k]) / (boundaries[k + 1] + boundaries[k])
        if r < gamma:
            gamma = r
    r = (np.pi - boundaries[Npic - 1]) / (np.pi + boundaries[Npic - 1])
    if r < gamma:
        gamma = r
    gamma = (1 - 1 / Nsig) * gamma  # this ensure that gamma is chosen as strictly less than the min

    mfb = np.zeros([Nsig, Npic + 1])

    # EWT_Meyer_Scaling
    Mi = int(np.floor(Nsig / 2))
    w = np.fft.fftshift(np.linspace(0, 2 * np.pi - 2 * np.pi / Nsig, num=Nsig))
    w[0:Mi] = -2 * np.pi + w[0:Mi]
    aw = abs(w)
    yms = np.zeros(Nsig)
    an = 1. / (2 * gamma * boundaries[0])
    pbn = (1. + gamma) * boundaries[0]
    mbn = (1. - gamma) * boundaries[0]
    for k in range(Nsig):
        if aw[k] <= mbn:
            yms[k] = 1
        elif ((aw[k] >= mbn) and (aw[k] <= pbn)):
            yms[k] = np.cos(np.pi * EWT_beta(an * (aw[k] - mbn)) / 2)
    yms = np.fft.ifftshift(yms)
    mfb[:, 0] = yms

    # generate rest of the wavelets
    for k in range(Npic - 1):
        mfb[:, k + 1] = EWT_Meyer_Wavelet(boundaries[k], boundaries[k + 1], gamma, Nsig)

    mfb[:, Npic] = EWT_Meyer_Wavelet(boundaries[Npic - 1], np.pi, gamma, Nsig)

    return mfb
def EWT_beta(x):
    """
    Beta = EWT_beta(x)
    function used in the construction of Meyer's wavelet
    """
    if x<0:
        bm=0
    elif x>1:
        bm=1
    else:
        bm=(x**4)*(35.-84.*x+70.*(x**2)-20.*(x**3))
    return bm
def EWT_Meyer_Wavelet(wn,wm,gamma,Nsig):
    """
    =========================================================
    ymw=EWT_Meyer_Wavelet(wn,wm,gamma,N)
    
    Generate the 1D Meyer wavelet in the Fourier
    domain associated to scale segment [wn,wm] 
    with transition ratio gamma
    
    Input parameters:
      -wn : lower boundary
      -wm : upper boundary
      -gamma : transition ratio
      -N : number of point in the vector
    
    Output:
      -ymw: Fourier transform of the wavelet on the band [wn,wm]
    Author: Jerome Gilles
    Institution: UCLA - Department of Mathematics
    Year: 2012
    Version: 1.0
    
    Python Version: Vinícius Rezende Carvalho - vrcarva@ufmg.br
    Universidade Federal de Minas Gerais - Brasil
    Núcleo de Neurociências 
    ==========================================================            
    """
    Mi=int(np.floor(Nsig/2))
    w=np.fft.fftshift(np.linspace(0,2*np.pi - 2*np.pi/Nsig,num = Nsig))
    w[0:Mi]=-2*np.pi+w[0:Mi]
    aw=abs(w)
    ymw=np.zeros(Nsig)
    an=1./(2*gamma*wn)
    am=1./(2*gamma*wm)
    pbn=(1.+gamma)*wn
    mbn=(1.-gamma)*wn
    pbm=(1.+gamma)*wm
    mbm=(1.-gamma)*wm

    for k in range(Nsig):
        if ((aw[k]>=pbn) and (aw[k]<=mbm)):
            ymw[k]=1
        elif ((aw[k]>=mbm) and (aw[k]<=pbm)):
            ymw[k]=np.cos(np.pi*EWT_beta(am*(aw[k]-mbm))/2)
        elif ((aw[k]>=mbn) and (aw[k]<=pbn)):
            ymw[k]=np.sin(np.pi*EWT_beta(an*(aw[k]-mbn))/2)
    ymw=np.fft.ifftshift(ymw)
    return ymw

In [10]:
my_files = glob.glob('*txt')

In [11]:
len(my_files)

101000

In [12]:
input_ = np.zeros((1000,12,10,126))

In [10]:
loudness_data_n2 = np.array([-0.767364,-0.476870,-0.193201,0.078374,0.172603,0.419829,0.500441,0.720262,0.920215,1.080807,1.295526,1.501556])


In [14]:
"""
Generation of Dataset(F1N2) using R1 1000 cycles
"""

'\nGeneration of Dataset(F1N2) using R1 1000 cycles\n'

In [15]:
x_train = []
y_train = []

In [34]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 1:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        if my_files[m][-6:-4]=='40':
            continue
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [17]:
boundaries = np.loadtxt('C:/users/vinee/Documents/BTP/n2_avg_r1/boundaries_n2.txt')

In [19]:
idx

[1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1455]

In [33]:
my_files[idx[1]][-6:-4]

'40'

In [35]:
len(x_train)

1000

In [44]:
x = np.array(x_train)

In [45]:
y = np.array(y_train)

In [39]:
x.shape

(1000, 12, 10, 126)

In [40]:
y.shape

(1000, 12)

In [41]:
x = list(x)

In [42]:
y = list(y)

In [46]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_x2_cnn_1',x)

In [47]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_y2_cnn_1',y)

In [48]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 2:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        if my_files[m][-6:-4]=='40':
            continue
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [49]:
len(x_train)

2000

In [51]:
len(y_train)

2000

In [52]:
x = np.array(x_train)

In [53]:
y = np.array(y_train)

In [54]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_x2_cnn_2',x)

In [55]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_y2_cnn_2',y)

In [56]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 3:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        if my_files[m][-6:-4]=='40':
            continue
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [57]:
len(x_train)

3000

In [58]:
y = np.array(y_train)

In [62]:
x = np.array(x_train)

In [63]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_x2_cnn_3',x)

In [64]:
np.save('C:/users/vinee/Documents/BTP/n2_avg_r1/ewt_y3_cnn_3',y)

In [1]:
# N2R4 for testing purposes

In [7]:
my_files = glob.glob('*txt')

In [8]:
len(my_files)

101000

In [20]:
x_train = []
y_train = []

In [12]:
boundaries = np.array([0.63679854007417,1.24043747184364,1.90051054875325,2.19052315291591,2.42542041587323,2.62882317400665,2.76529989452493,2.88996614389811,3.01463240497312])

In [13]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 4 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [ ]:
for j in range(1,1001,1):
    idx = []
    for i in range(len(my_files)):
        string1 = my_files[i]
        p = list(map(int, re.findall(r'\d+', string1)))
        if p[-2] == j and p[-3] == 4 and int(p[-1])>40:
            idx.append(i)
    res = np.zeros((12,10,126))
    i  = 0
    for m in idx:
        file = np.loadtxt(my_files[m])
        ewt = ewt_vineet(file,boundaries)
        ewt = ewt.transpose()
        res[i] = ewt
        i+=1
    x_train.append(res)
    y_train.append(loudness_data_n2)

In [22]:
idx = []
for i in range(len(my_files)):
    string1 = my_files[i]
    p = list(map(int, re.findall(r'\d+', string1)))
    if p[-2] == j and p[-3] == 4 and int(p[-1])>40:
        idx.append(i)

In [24]:
len(idx)

12

In [14]:
len(x_train)

1000

In [15]:
len(y_train)

1000

In [16]:
x = np.array(x_train)
y = np.array(y_train)

In [17]:
x.shape

(1000, 12, 10, 126)

In [18]:
y.shape

(1000, 12)

In [19]:
np.save('C:/Users/vinee/Documents/BTP/x_new_n2',x)
np.save('C:/Users/vinee/Documents/BTP/y_new_n2',y)

In [21]:
pwd

'C:\\Users\\vinee\\Documents\\BTP\\resampled_dataset_f1_n2'